
# 📊 Offline Search Evaluation on Amazon ESCI

This notebook implements an **offline evaluation prototype** for e-commerce search using the **Amazon ESCI** dataset.
- Dataset: [amazon-science/esci-data](https://github.com/amazon-science/esci-data)
- Algorithms: BM25 (keyword-based) and SBERT+FAISS (semantic)
- Metrics: nDCG@K, MAP@K, MRR, Precision/Recall@K


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Install dependencies
%%capture
!pip -q install pandas numpy scikit-learn rank_bm25 sentence-transformers faiss-cpu matplotlib tqdm

In [ ]:
import os, random, math
import numpy as np, pandas as pd
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
import faiss
import matplotlib.pyplot as plt
from collections import defaultdict
from math import log2
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from scipy import stats

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [ ]:
import pandas as pd
df_examples = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/Datasets/amazon_esci/shopping_queries_dataset_examples.parquet')
df_products = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/Datasets/amazon_esci/shopping_queries_dataset_products.parquet')

for c in ["product_title","product_bullet_point","product_brand"]:
    if c in df_products.columns:
        df_products[c] = df_products[c].fillna("").astype(str)
df_products["doc_text"] = df_products[["product_title","product_bullet_point","product_brand"]].agg(" ".join, axis=1)
df_products = df_products[["product_id","doc_text", "product_locale"]].drop_duplicates()

df_examples_products = pd.merge(
    df_examples,
    df_products,
    how='left',
    left_on=['product_locale','product_id'],
    right_on=['product_locale', 'product_id']
)
df_task_1 = df_examples_products[df_examples_products["small_version"] == 1]
df_task_1 = df_task_1[df_task_1["product_locale"] == "us"]
df_task_1.head(n=30)

In [ ]:
df_task_1.rename(columns={"esci_label":"relevance_label"}, inplace=True)
df_products = df_products[["product_id","doc_text"]].drop_duplicates()
pairs_all = df_task_1[["query","product_id","relevance_label","doc_text"]].drop_duplicates()
print("Total pairs:", pairs_all.shape)

In [ ]:
GRADE = {"E":3,"S":2,"C":1,"I":0}
pairs_all = pairs_all[pairs_all["relevance_label"].isin(GRADE.keys())].copy()
queries = pairs_all["query"].unique().tolist()
random.shuffle(queries)
queries = queries[:500]  # sample for runtime
pairs_eval = pairs_all[pairs_all["query"].isin(queries)]
print("Eval queries:", len(queries))

In [ ]:
tokenized_corpus = [doc.split() for doc in df_products["doc_text"].tolist()]
bm25 = BM25Okapi(tokenized_corpus)

embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
doc_embeddings = embed_model.encode(df_products["doc_text"].tolist(), batch_size=256, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)

dim = doc_embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(doc_embeddings)

pid_to_row = {pid:i for i,pid in enumerate(df_products["product_id"].tolist())}
row_to_pid = {i:pid for pid,i in pid_to_row.items()}

In [ ]:
def retrieve_bm25(query, topk=100):
    tokens = query.split()
    scores = bm25.get_scores(tokens)
    idx = np.argpartition(scores, -topk)[-topk:]
    idx = idx[np.argsort(scores[idx])[::-1]]
    # Updated to directly use indices to get product_id from df_products
    return [(df_products.iloc[i]["product_id"], float(scores[i])) for i in idx]

def retrieve_sbert(query, topk=100):
    q_emb = embed_model.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    scores, I = index.search(q_emb.astype(np.float32), topk)
    # Updated to directly use indices to get product_id from df_products
    return [(df_products.iloc[int(i)]["product_id"], float(s)) for i,s in zip(I[0], scores[0])]

In [ ]:
def make_qrels(pairs):
    qrels = defaultdict(dict)
    for _,r in pairs.iterrows():
        qrels[r["query"]][r["product_id"]] = GRADE[r["relevance_label"]]
    return qrels

QRELS = make_qrels(pairs_eval)

def dcg_at_k(gains,k):
    return sum(g/log2(i+2) for i,g in enumerate(gains[:k]))
def ndcg_at_k(ranked,qrels,k):
    gains = [qrels.get(pid,0) for pid in ranked]
    dcg = dcg_at_k(gains,k)
    idcg = dcg_at_k(sorted(qrels.values(),reverse=True),k)
    return dcg/idcg if idcg>0 else 0

def precision_at_k(ranked,relset,k):
    topk = ranked[:k]
    return sum(pid in relset for pid in topk)/max(1,len(topk))
def recall_at_k(ranked,relset,k):
    topk = ranked[:k]
    return sum(pid in relset for pid in topk)/max(1,len(relset)) if relset else 0
def average_precision_at_k(ranked,relset,k):
    hits,ap=0,0.0
    for i,pid in enumerate(ranked[:k],start=1):
        if pid in relset:
            hits+=1; ap+=hits/i
    return ap/max(1,len(relset)) if relset else 0
def mrr_at_k(ranked,relset,k):
    for i,pid in enumerate(ranked[:k],start=1):
        if pid in relset: return 1.0/i
    return 0.0

In [ ]:
def evaluate(retrieve_fn,qrels,k=20):
    rows=[]
    for q in tqdm(qrels.keys()):
        graded=qrels[q]
        exact={pid for pid,g in graded.items() if g==3}
        anyrel={pid for pid,g in graded.items() if g>0}
        ranked=[pid for pid,_ in retrieve_fn(q,topk=max(k,100))]
        rows.append({
            "query":q,
            "ndcg@k":ndcg_at_k(ranked,graded,k),
            "map@k":average_precision_at_k(ranked,anyrel,k),
            "mrr@k":mrr_at_k(ranked,anyrel,k),
            "p@k_exact":precision_at_k(ranked,exact,k),
            "r@k_exact":recall_at_k(ranked,exact,k),
            "p@k_any":precision_at_k(ranked,anyrel,k),
            "r@k_any":recall_at_k(ranked,anyrel,k),
        })
    return pd.DataFrame(rows)

metrics_B = evaluate(retrieve_sbert,QRELS,k=20)
metrics_A = evaluate(retrieve_bm25,QRELS,k=20)
summary = pd.concat([metrics_A.mean(numeric_only=True),metrics_B.mean(numeric_only=True)],axis=1)
summary.columns=["BM25","SBERT"]
summary

In [ ]:
def paired_stats(a,b):
    diff=(b-a).dropna().values
    t,p=stats.ttest_rel(b,a,nan_policy="omit")
    boots=[resample(diff).mean() for _ in range(1000)]
    lo,hi=np.percentile(boots,[2.5,97.5])
    return {"delta_mean":diff.mean(),"p_value":p,"boot95":[lo,hi]}

joined=metrics_A.set_index("query").join(metrics_B.set_index("query"),lsuffix="_A",rsuffix="_B")
results={}
for m in ["ndcg@k","map@k","mrr@k"]:
    results[m]=paired_stats(joined[f"{m}_A"],joined[f"{m}_B"])
results

In [ ]:
delta = joined["ndcg@k_B"]-joined["ndcg@k_A"]
plt.hist(delta,bins=30)
plt.title("ΔnDCG@20 (B−A)")
plt.show()